### Ingest constructors.json file

In [ ]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [ ]:
from formula1.formula1_constants import *
from formula1.formula1_utils import *

##### Step 1 - Read the JSON file using the spark dataframe reader

In [ ]:
from pyspark.sql.functions import lit

constructors_schema = "constructorId INT, constructorRef STRING, name STRING, nationality STRING, url STRING"

# Read from Landing Zone
constructor_df = spark.read.schema(constructors_schema).json(
    f"{landing_folder_path}/{v_file_date}/constructors.json"
)

##### Step 2 - Add ingestion meta data

In [ ]:
constructor_final_df = (
    add_ingestion_date(constructor_df)
    .withColumn("data_source", lit(v_data_source))
    .withColumn("file_date", lit(v_file_date))
)

##### Step 3 Write output to bronze container

In [ ]:
constructor_final_df.write.mode("overwrite").format("delta").saveAsTable(
    "f1_bronze.constructors"
)